In [41]:
from pycorenlp import StanfordCoreNLP
import json

nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

def get_ner(sentence):
    result = dependency_parser(sentence, properties={
        "outputFormat": "json",
        "annotators": "ner"})
    res = []
    for obj in result['sentences'][0]['entitymentions']:
        res.append((obj['text'], obj['ner']))
    return res

In [42]:
get_ner("you owe obama $4 in indonesia")

[('$4', 'MONEY'), ('indonesia', 'COUNTRY')]

In [60]:
import pandas as pd
from tqdm import tqdm_notebook

def run(tipe):
    df = pd.read_csv("res_mul_all.csv")
    sf = pd.DataFrame(columns = ['id','review','category','term'])
    count = 0
    all_ner = []
    for index in tqdm_notebook(range(len(df))):
        new_term = []
        ner = get_ner(df['review'][index])
        if len(ner):
            print(df['review'][index])
            print(ner)
            print()
            for word, tipe in ner:
                all_ner.append(word)
    return all_ner
#         for nn in get_nouns(df['review'][index]):
#             new_term.append(nn)
#         if len(new_term) == 0:
#             new_term = '|'.join(get_nouns(df['review'][index]))
#             count += 1
        
#         sf = sf.append({'id': df['id'][index], 
#                     'review': df['review'][index],
#                     'category': df['category'][index],
#                     'term': '|'.join(new_term)
#                    }, ignore_index=True)
#     print(count)

#     sf.to_csv("Results/add_noun/"+ tipe +".csv")
#     sf.to_excel("Results/add_noun/"+ tipe +".xlsx")

In [62]:
new_ner = run('20')

HBox(children=(IntProgress(value=0, max=699), HTML(value='')))

We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.
[('four', 'NUMBER'), ('noon', 'TIME')]

I have eaten at Saul, many times, the food is always consistently, outrageously good.
[('Saul', 'PERSON')]

The food is very average...the Thai fusion stuff is a bit too sweet, every thing they serve is too sweet here.
[('Thai', 'NATIONALITY')]

The decor is night tho...but they REALLY need to clean that vent in the ceiling...its quite un-appetizing, and kills your effort to make this place look sleek and modern.
[('night', 'TIME')]

We went around 9:30 on a Friday and it had died down a bit by then so the service was great!
[('9:30', 'TIME'), ('Friday', 'DATE')]

i happen to have a policy that goes along with a little bit of self-respect, which includes not letting a waiter intimidate me, i.e. make me feel bad asking for trivialities like water, or the check.
[('waiter', 'TITLE')]

We took advantage of the

Try the Pad Thai, it's fabulous and their prices are so cheap!
[('Thai', 'NATIONALITY')]

Also, the sandwiches (nearing $7) didn't come with anything like chips or a side.
[('$7', 'MONEY')]

They have authentic Indian at amazin prices.
[('Indian', 'NATIONALITY')]

The lobster sandwich is $24 and although it was good it was not nearly enough to warrant that price.
[('$24', 'MONEY')]

Truly the mark of an attentive waiter.
[('waiter', 'TITLE')]

We ate at this Thai place following the reviews but very unhappy with the foods.
[('Thai', 'NATIONALITY')]

Have frequented 'ino for several years and the food remains excellent.
[('several years', 'DURATION')]

Cheese plate is a varied delight and great bargain at $10.
[('$10', 'MONEY')]

The wait staff is very freindly, they make it feel like you're eating in a freindly little european town.
[('european', 'NATIONALITY')]

After dinner the manager grabbed my boyfriend, asked him: Where are you from...maybe you dont know how things work in Americ

Went here last night - nice decor, good service, but the food was surprisingly excellent.
[('last night', 'TIME')]

Best Taiwanese food in NY!
[('Taiwanese', 'NATIONALITY'), ('NY', 'STATE_OR_PROVINCE')]

Rao's has the best service and atmosphere in NYC.
[('Rao', 'PERSON'), ('NYC', 'CITY')]

We recently decided to try this location, and to our delight, they have outdoor seating, perfect since I had my yorkie with me.
[('recently', 'DATE')]

I got the $10 10-piece dim sum combo, every bite of which was great.
[('$10', 'MONEY')]

Great Indian food and the service is incredible.
[('Indian', 'NATIONALITY')]

Try the congee and the donut like deep fried dough they call Ow Ley Soh, a delicious and sweet tasting bread.
[('Ley Soh', 'PERSON')]

Simply some good tasting Chinese food at incredible prices...
[('Chinese', 'NATIONALITY')]

Consistently good Japanese Tapas.
[('Japanese', 'NATIONALITY'), ('Tapas', 'MISC')]

Yakitori (bbq meats) is tasty too.
[('Yakitori', 'LOCATION')]

We were seated 

In [71]:
splitted_ner = []
for ner in new_ner:
    for word in ner.split(' '):
        splitted_ner.append(word.lower())

In [73]:
ner_ner = sorted(list(dict.fromkeys(splitted_ner)))

In [75]:
print(ner_ner)

['$*', '$10', '$18', '$24', '$3-6', '$6', '$7', '$8.00', '$8.50', '100', '13', '15', '1st', '2', '20', '2nd', '2\xa01/2', '4', '5', '50', '6th', '7', '8th', '9:30', 'a', 'ago', 'america', 'authentic', 'authority', 'bart', 'bartender', 'best', 'brooklyn', 'bukhara', 'chef', 'chinatown', 'chinese', 'city', 'citysearch', 'cobb', 'couple', 'dal', 'day', 'dining', 'dj', 'dollar', 'earlier', 'emilio', 'england', 'european', 'evening', 'every', 'few', 'first', 'four', 'french', 'friday', 'garden', 'gulab', 'he', 'her', 'him', 'his', 'host', 'hostess', 'hudson', 'indian', 'italian', 'italy', 'jamun', 'japan', 'japanese', 'kha', 'la', 'lanterna', 'lasagna', 'last', 'late', 'ley', 'lobster', 'manager', 'manhattan', 'menu', 'minutes', 'mizu', 'months', "mother's", 'new', 'night', 'nights', 'noon', 'northern', 'now', 'ny', 'nyc', 'of', 'once', 'one', 'onion', 'owner', 'pakistani', 'paris', 'parisian', 'past', 'port', 'present', 'raga', 'rao', 'recently', 'reviewer', 'river', 'roxy', 'rubber', 'sal

In [84]:
import pandas as pd

def run_filter(tipe, where, ner):
    df = pd.read_csv("Results/" + where + tipe +".csv")
#     print(df.head())
    sf = pd.DataFrame(columns = ['id','review','category','term'])
    for index, concat_terms in enumerate(df['term']):
        new_terms = []
        try:
            terms = concat_terms.split('|')
        except:
            terms = []
        for term in terms:
            if term not in ner and len(term) > 3 and "thing" not in term and "time" not in term:
                new_terms.append(term)
        sf = sf.append({'id': df['id'][index], 
                        'review': df['review'][index],
                        'category': df['category'][index],
                        'term': '|'.join(new_terms)
                       }, ignore_index=True)

    sf.to_csv("Results/Filter/"+ tipe +"-ner-ex.csv")
    sf.to_excel("Results/Filter/"+ tipe +"-ner-ex.xlsx")

In [85]:
run_filter("20", 'Filter/', ner_ner)